# DATA20001 Deep Learning - Group Project
## Image project

**Due Wednesday December 13, before 23:59.**

The task is to learn to assign the correct labels to a set of images.  The images are originally from a photo-sharing site and released under Creative Commons-licenses allowing sharing.  The training set contains 20 000 images. We have resized them and cropped them to 128x128 to make the task a bit more manageable.

We're only giving you the code for downloading the data. The rest you'll have to do yourselves.

Some comments and hints particular to the image project:

- One image may belong to many classes in this problem, i.e., it's a multi-label classification problem. In fact there are images that don't belong to any of our classes, and you should also be able to handle these correctly. Pay careful attention to how you design the outputs of the network (e.g., what activation to use) and what loss function should be used.

- As the dataset is pretty imbalanced, don't focus too strictly on the outputs being probabilistic. (Meaning that the right threshold for selecting the label might not be 0.5.)

- Image files can be loaded as numpy matrices for example using `imageio.imread`. Most images are color, but a few grayscale. You need to handle the grayscale ones somehow as they would have a different number of color channels (depth) than the color ones.

- Loading all the images into one big matrix as we have done in the exercises is not feasible (e.g. the virtual servers in CSC have only 3 GB of RAM). You need to load the images in smaller chunks for the training. This shouldn't be a problem we are doing mini-batch training anyway, and thus we don't need to keep all the images in memory. You can simply pass you current chunk of images to `model.fit()` as it remembers the weights from the previous run.

- You need to think carefully about how you load the annotations and match them up with the corresponding images, especially as you are loading them in smaller chunks.

## Download the data

In [ ]:
from keras.utils.data_utils import get_file

database_path = 'train/'

dl_file='dl2017-image-proj.zip'
dl_url='https://www.cs.helsinki.fi/u/mvsjober/misc/'
get_file(dl_file, dl_url+dl_file, cache_dir='./', cache_subdir=database_path, extract=True)

The above command downloaded and extracted the data files into the `train` subdirectory.

The images can be found in `train/images`, and are named as `im1.jpg`, `im2.jpg` and so on until `im20000.jpg`.

The class labels, or annotations, can be found in `train/annotations` as `CLASSNAME.txt`, where CLASSNAME is one of the fourteen classes: *baby, bird, car, clouds, dog, female, flower, male, night, people, portrait, river, sea,* and *tree*.

Each annotation file is a simple text file that lists the images that depict that class, one per line. The images are listed with their number, not the full filename. For example `5969` refers to the image `im5969.jpg`.

## Your stuff goes here ...

### Import

In [ ]:
%matplotlib inline

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.layers.convolutional import *
from keras.preprocessing.image import *
from keras import backend as K
from keras.utils.vis_utils import model_to_dot
from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
from scipy import misc
from sklearn.metrics import f1_score
from IPython.display import SVG

### Pre-process

In [ ]:
class_indices = {0:"baby",
                 1:"bird",
                 2:"car",
                 3:"clouds",
                 4:"dog",
                 5:"female",
                 6:"flower",
                 7:"male",
                 8:"night",
                 9:"people",
                 10:"portrait",
                 11:"river",
                 12:"sea",
                 13:"tree"}

#### Label and file name loading

In [ ]:
N = 20000
num_class = len(class_indices)


img_list = [database_path+"images/"+"im"+str(x)+".jpg" for x in range(1,N+1)]
y = np.zeros((N,num_class))

for k,v in class_indices.items():
    with open(database_path+"annotations/"+v+".txt","r") as fp:
        for line in fp:
            y[int(line)-1,k] = 1

#### Loading dataset

In [ ]:
img_size = (32,32)

image_load_func = lambda path: misc.imresize(misc.imread(path,mode="RGB"),size=img_size)

In [ ]:
example=3
img = image_load_func(img_list[example])
for l in range(3):
    plt.subplot(1, 4, l+1)
    plt.imshow(img[:,:,l])
    plt.axis('off')
plt.subplot(1, 4, 4)
plt.imshow(img)
plt.axis('off')

if sum(y[example]==1)==0:
    print("No label")
else:
    for i in np.argwhere(y[example]==1).flatten():
        print(class_indices[i])

In [ ]:
X = np.stack([image_load_func(file) for file in img_list])
print(X.shape)

### Model Building

In [ ]:
from densenet import *

In [ ]:
def getThreshold(y_pred,y_true,threshold_interval):
    threshold = np.arange(0,1,threshold_interval)

    best_threshold = np.zeros(y_true.shape[1])
    for i in range(y_pred.shape[1]):
        temp = np.array([[1 if pred>j else 0 for j in threshold] for pred in y_pred[:,i]])
        score = np.array([f1_score(y_true[:,i],temp[:,j], average='micro') for j in range(len(threshold)) ])
        best_threshold[i] = threshold[score.argmax()]
        print("Best threshold for class",i,"：",best_threshold[i])
    return best_threshold

#### Standard DenseNet with output layer adjusted

In [ ]:
nb_filter=32
weight_decay=0.00001
dropout_rate=0.2
nb_dense_block=3
nb_layers=2
growth_rate=12

dn = DenseNet(num_class, X.shape[1:], nb_layers*3+4, nb_dense_block, growth_rate,
             nb_filter, dropout_rate=None, weight_decay=1E-4)
dn.compile(loss='binary_crossentropy', 
              optimizer='adam')

print(dn.summary())

In [ ]:
SVG(model_to_dot(dn, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
%%time

epochs = 3

history = dn.fit(X,y,epochs=epochs,batch_size=64,verbose=1)

In [ ]:
y_pred = dn.predict(X[500:530])
print(y_pred)

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['loss'])
plt.title('loss')

In [ ]:
%%time

nb_filter=32
weight_decay=0.00001
dropout_rate=0.2
nb_dense_block=3
nb_layers=2
growth_rate=12

epochs = 1
#5-fold cross validation
score = []
kf = KFold(n_splits=5,shuffle=True)
for train,val in kf.split(X, y):
    dn = DenseNet(num_class, X.shape[1:], nb_layers*3+4, nb_dense_block, growth_rate,
             nb_filter, dropout_rate=None, weight_decay=1E-4)
    dn.compile(loss='binary_crossentropy', optimizer='adam')
    dn.fit(X[train],y[train],epochs=epochs,batch_size=64,verbose=1)
    y_train_pred = dn.predict(X[train],batch_size=32, verbose=1)
    threshold=getThreshold(y_train_pred,y[train],0.001)
    y_val_pred = 1*(dn.predict(X[val],batch_size=32, verbose=1)>threshold)    
    score.append(f1_score(y[val], y_val_pred, average='micro'))
print("F1 Score:", np.mean(score),"+/-", np.std(score))

#### Customized DenseNet

In [ ]:
def getCutmoizedDenseNet():
    nb_filter=32
    weight_decay=0.00001
    dropout_rate=0.2
    nb_dense_block=3
    nb_layers=2
    growth_rate=1
    
    model_input = Input(shape=X.shape[1:])

    x = Conv2D(nb_filter, (3, 3),
                   kernel_initializer="he_uniform",
                   padding="same",
                   name="initial_conv2D",
                   use_bias=True,
                   kernel_regularizer=l2(weight_decay))(model_input)
    #blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = denseblock(x, nb_layers, nb_filter, growth_rate,
                                  dropout_rate=dropout_rate)
        # add transition
        x = transition(x, nb_filter, dropout_rate=dropout_rate,
                       weight_decay=weight_decay)

    #last block 
    x, nb_filter = denseblock(x, nb_layers, nb_filter, growth_rate,
                                  dropout_rate=dropout_rate,
                                  weight_decay=weight_decay)

    x = BatchNormalization(axis=1,
                           gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D(data_format=K.image_data_format())(x)
    #more layers

    model_output = Dense(num_class, activation='sigmoid')(x)

    model = Model(inputs=[model_input], outputs=[model_output], name="DenseNet")
    
    return model

In [ ]:
cdn=getCutmoizedDenseNet()
cdn.compile(loss='binary_crossentropy', optimizer='adam')

print(cdn.summary())

In [ ]:
SVG(model_to_dot(cdn, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
%%time

epochs = 3

history = cdn.fit(X,y,epochs=epochs,batch_size=64,verbose=1)

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['loss'])
plt.title('loss')

In [ ]:
%%time

nb_filter=32
weight_decay=0.00001
dropout_rate=0.2
nb_dense_block=3
nb_layers=2
growth_rate=1

epochs = 3
#5-fold cross validation
score = []
kf = KFold(n_splits=5,shuffle=True)
for train,val in kf.split(X, y):
    cdn=getCutmoizedDenseNet()
    cdn.compile(loss='binary_crossentropy', optimizer='adam')
    cdn.fit(X[train],y[train],epochs=epochs,batch_size=64,verbose=1)
    y_train_pred = cdn.predict(X[train],batch_size=32, verbose=1)
    threshold=getThreshold(y_train_pred,y[train],0.001)
    y_val_pred = 1*(cdn.predict(X[val],batch_size=32, verbose=1)>threshold)    
    score.append(f1_score(y[val], y_val_pred, average='micro'))
print("F1 Score:", np.mean(score),"+/-", np.std(score))

#### Classic ConvNet

In [ ]:
def getConvNet():
    model=Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3),padding='same', input_shape=X.shape[1:],activation="relu"))
    model.add(Conv2D(filters=32, kernel_size=(3, 3),activation="relu"))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=64, kernel_size=(3, 3),activation="relu"))
    model.add(Conv2D(filters=64, kernel_size=(3, 3),activation="relu"))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    model.add(Dense(num_class))
    model.add(Activation("sigmoid"))
    return model

In [ ]:
cn=getConvNet()
cn.compile(loss='binary_crossentropy',optimizer='adam')
print(cn.summary())

In [ ]:
%%time

epochs = 3

history = cn.fit(X,y,epochs=epochs,batch_size=64,verbose=1)

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['loss'])
plt.title('loss')

In [ ]:
%%time


epochs = 1
#5-fold cross validation
score = []
kf = KFold(n_splits=5,shuffle=True)
for train,val in kf.split(X, y):
    cn=getConvNet()
    cn.compile(loss='binary_crossentropy', optimizer='adam')
    cn.fit(X[train],y[train],epochs=epochs,batch_size=64,verbose=1)
    y_train_pred = cn.predict(X[train],batch_size=32, verbose=1)
    threshold=getThreshold(y_train_pred,y[train],0.001)
    y_val_pred = 1*(cn.predict(X[val],batch_size=32, verbose=1)>threshold)    
    score.append(f1_score(y[val], y_val_pred, average='micro'))
print("F1 Score:", np.mean(score),"+/-", np.std(score))

## Save your model

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [ ]:
model.save('model.h5')

The model file should now be visible in the "Home" screen of the jupyter notebooks interface.  There you should be able to select it and press "download".

## Predict for test set

You will be asked to return your prediction for the testset.  These should be returned as a matrix with one row for each test set image.  Each row contains a binary prediction for each label, 1 if it's present in the image, and 0 if not. The order of the labels is as follows (alphabetic order of the label names):

    baby bird car clouds dog female flower male night people portrait river sea tree

An example row could like like this if your system predicts the presense of a bird and clouds:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0
    
If you have the matrix prepared in `y` (e.g., by calling `y=model.predict(x_test)`) you can use the following function to save it to a text file.

In [ ]:
np.savetxt('results.txt', y, fmt='%d')